# Food Recommendation System

## Preparation

**Import libary**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

**Import data**

Make sure that these datasets have already in the same directory as this notebook. For Google Colab users, please upload these datasets to Files.


*   foods.csv 
*   users.csv
*   users_foods_.csv



In [ ]:
foods = pd.read_csv('foods.csv')
users = pd.read_csv('users.csv')
users_foods = pd.read_csv('users_foods.csv')

Foods dataset shows all foods in the system and the compability to diet types.

In [ ]:
foods.head()

,food_id,diet_1,diet_2,diet_3,diet_4,diet_5
0,food_1,0,0,1,1,1
1,food_2,0,0,1,1,0
2,food_3,0,0,1,1,1
3,food_4,0,0,1,0,1
4,food_5,0,1,0,0,1


Users dataset shows all users in the system and their respective diet type

In [ ]:
users.head()

,user_id,diet_type
0,user_1,diet_3
1,user_2,diet_1
2,user_3,diet_1
3,user_4,diet_3
4,user_5,diet_4


Users & Foods dataset shows rating that users have given to the foods.

Rating 0 means that they didn't give rating.
Rating 1 means they don't like the food and rating 5 means they like the food.

In [ ]:
users_foods.head()

,user_id,food_1,food_2,food_3,food_4,food_5,food_6,food_7,food_8,food_9,food_10,food_11,food_12,food_13,food_14,food_15,food_16,food_17,food_18,food_19,food_20
0,user_1,0,3,0,5,0,2,2,0,4,3,0,5,0,5,0,4,1,3,0,0
1,user_2,2,5,3,0,1,0,0,4,5,0,3,0,0,0,2,5,1,5,3,3
2,user_3,3,0,5,1,5,5,0,3,0,2,0,4,0,0,2,5,3,1,1,0
3,user_4,5,0,0,4,2,5,0,2,2,3,1,0,0,0,4,3,0,0,4,0
4,user_5,0,3,4,0,2,5,5,4,1,4,5,0,2,3,0,0,0,2,0,0


## Select user to give recommendation to

Please refer to the list below for available user ID(s).

In [ ]:
print(list(users['user_id']))

['user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6', 'user_7', 'user_8', 'user_9', 'user_10', 'user_11', 'user_12', 'user_13', 'user_14', 'user_15', 'user_16', 'user_17', 'user_18', 'user_19', 'user_20']


Input exactly 1 user ID to the field below. Don't use singe quote. To avoid misspelling, consider using copy & paste.

Run the cell before continuing

In [ ]:
#@title Select User
user_to_recommend = "user_18" #@param {type:"string"}


## Prepare food items to recommend

**Take rating of foods that selected user has given**

In [ ]:
foods_rating_by_user = users_foods[users_foods['user_id'] == user_to_recommend].T.reset_index().drop(0)

foods_rating_by_user.head()

,index,17
1,food_1,0
2,food_2,2
3,food_3,0
4,food_4,0
5,food_5,5


**Take food items that selected user likes**

We can assume that user likes the food if they give 4 or 5 rating to the food.

Food that user have liked:

In [ ]:
user_index = list(users['user_id']).index(user_to_recommend)
foods_liked_by_user = list(foods_rating_by_user[foods_rating_by_user[user_index] >= 4]['index'])

print(foods_liked_by_user)

['food_5', 'food_8', 'food_9', 'food_12', 'food_14']


Food that user haven't given rated to:

In [ ]:
foods_never_rated = list(foods_rating_by_user[foods_rating_by_user[user_index] == 0]['index'])

print(foods_never_rated)

['food_1', 'food_3', 'food_4', 'food_6', 'food_7', 'food_13', 'food_16', 'food_19', 'food_20']


Filter food that user haven't rated by user's diet type

In [ ]:
user_diet_type = list(users[users['user_id'] == user_to_recommend]['diet_type'])[0]

foods_never_rated = foods.set_index('food_id').filter(items=foods_never_rated, axis=0)
is_good_for_diet = foods_never_rated[user_diet_type] == 1
foods_can_recommend = list(foods_never_rated[is_good_for_diet].reset_index()['food_id'])

print(foods_can_recommend)

['food_7', 'food_13', 'food_16', 'food_20']


## Count recommendation

To count recommendation, we calculate similarity between food items that user has already liked and foods that is possible to be recommended (foods that user hasn't rated and compatible to user's diet type). We use item-based collaborative filtering with cosine similarity algorithm to do this.

In [ ]:
recommended_food = {}

for food_liked in foods_liked_by_user:
    food_liked_array = np.array(list(users_foods[food_liked])).reshape(1, -1)
    for food_to_recommend in foods_can_recommend:
        food_to_recommend_array = np.array(list(users_foods[food_to_recommend])).reshape(1, -1)
        score = cosine_similarity(food_liked_array, food_to_recommend_array)[0][0]
        if food_to_recommend in recommended_food:
            if (score > recommended_food[food_to_recommend]):
                recommended_food[food_to_recommend] = score
        else:
            recommended_food[food_to_recommend] = score
recommended_food = sorted(recommended_food.items(), key=lambda x: x[1], reverse=True)

print(recommended_food)

[('food_20', 0.6564215544231429), ('food_13', 0.5735770712514149), ('food_16', 0.5125589535607086), ('food_7', 0.4707565417620042)]
